# 🏠 DeepRoof-2026: Multi-Task Training Notebook

Welcome to the official training environment for the **DeepRoof-2026 AI Roof Layout Engine**. 

### 🛠 Step 1: Initialize & Fix Environment (Universal Fix)
This cell resolves path issues, installs the correct **MMCV binary**, provides missing **CUDA libraries**, and repairs `mmsegmentation`.

In [ ]:
import os
import sys
import subprocess
import torch
from pathlib import Path

# --- 1. SET UP PATHS ---
project_root = str(Path(os.getcwd()).parent)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"✅ Added {project_root} to sys.path")

# --- 2. NUCLEAR RECOVERY FOR MMSEGMENTATION ---
def recover_mmseg():
    print("🔍 Locating mmsegmentation...")
    mmseg_path = ""
    try:
        # Check typical Linux path first
        linux_path = "/usr/local/lib/python3.11/dist-packages/mmseg/__init__.py"
        if os.path.exists(linux_path):
            mmseg_path = linux_path
        else:
            # Fallback to pip show
            result = subprocess.check_output([sys.executable, "-m", "pip", "show", "mmsegmentation"], stderr=subprocess.DEVNULL).decode()
            for line in result.split('\n'):
                if line.startswith('Location: '):
                    mmseg_path = os.path.join(line.split(': ')[1].strip(), "mmseg/__init__.py")
                    break
    except Exception as e:
        print(f"❌ Error locating mmsegmentation: {e}")

    if mmseg_path and os.path.exists(mmseg_path):
        with open(mmseg_path, 'r') as f:
            if "OVERRIDE by DeepRoof" in f.read():
                print("✅ mmsegmentation is already patched/clean.")
            else:
                print(f"📍 Found corrupted/clean file at: {mmseg_path}")
                print("🩹 OVERWRITING with Clean Version (No Assertions)...")
                
                clean_content = """# Copyright (c) OpenMMLab. All rights reserved.
import mmcv
import mmengine
from mmengine.utils import digit_version

from .version import __version__, version_info

MMCV_MIN = '2.0.0rc4'
MMCV_MAX = '2.2.0'
MMENGINE_MIN = '0.7.1'
MMENGINE_MAX = '1.0.0'

mmcv_min_version = digit_version(MMCV_MIN)
mmcv_max_version = digit_version('9.9.9') # OVERRIDE by DeepRoof
mmcv_version = digit_version(mmcv.__version__)

mmengine_min_version = digit_version(MMENGINE_MIN)
mmengine_max_version = digit_version('9.9.9') # OVERRIDE by DeepRoof
mmengine_version = digit_version(mmengine.__version__)

__all__ = ['__version__', 'version_info', 'digit_version']
"""
                try:
                    with open(mmseg_path, 'w') as f:
                        f.write(clean_content)
                    print("✅ SUCCESSFULLY REPAIRED mmseg/__init__.py")
                except Exception as e:
                     print(f"❌ Failed to write file: {e}")
             
    # 3. Ensure Dependencies & MMCV Compatibility
    cuda_available = torch.cuda.is_available()
    print(f"🔍 CUDA Available: {cuda_available}")
    
    # --- CHECK FOR BROKEN LIBCUDART ---
    if cuda_available:
        try:
            import mmcv
            from mmcv.ops import point_sample # Trigger the load
            print("✅ MMCV Ops loaded successfully.")
        except ImportError as e:
            if "libcudart.so" in str(e):
                print("⚠️ Missing CUDA Runtime libraries (libcudart). Installing nvidia-cuda-runtime-cu11...")
                subprocess.check_call([sys.executable, "-m", "pip", "install", "nvidia-cuda-runtime-cu11"])
                
                # Add to path dynamically
                import nvidia.cuda_runtime.lib
                lib_dir = os.path.dirname(nvidia.cuda_runtime.lib.__file__)
                if lib_dir not in os.environ.get('LD_LIBRARY_PATH', ''):
                    os.environ['LD_LIBRARY_PATH'] = lib_dir + os.pathsep + os.environ.get('LD_LIBRARY_PATH', '')
                    print(f"✅ Added {lib_dir} to LD_LIBRARY_PATH")
                return False # Restart needed to reload libs
            elif "No module named 'mmcv._ext'" in str(e):
                 print("⚠️ MMCV-Lite detected on CUDA machine. Upgrading to MMCV-Full...")
                 subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "mmcv"])
                 subprocess.check_call([sys.executable, "-m", "pip", "install", "mmcv==2.2.0", "-f", "https://download.openmmlab.com/mmcv/dist/cu118/torch2.1/index.html"])
                 return False

    if not cuda_available:
        try:
            import mmcv
            from mmcv.ops import point_sample
            print("⚠️ CPU Environment detected but MMCV-Full (Ops) is installed. Downgrading...")
            subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "mmcv"])
            subprocess.check_call([sys.executable, "-m", "pip", "install", "mmcv==2.2.0"])
            return False
        except:
            pass

    for pkg in ["ftfy", "regex", "rasterio", "geopandas", "albumentations"]:
        try:
            subprocess.check_output([sys.executable, "-m", "pip", "show", pkg], stderr=subprocess.DEVNULL)
        except:
            print(f"📦 Installing {pkg}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
    
    print("✅ Environment dependencies checked.")
    return True

if recover_mmseg():
    # Force LD_LIBRARY_PATH update if needed before import
    try:
        import nvidia.cuda_runtime.lib
        lib_dir = os.path.dirname(nvidia.cuda_runtime.lib.__file__)
        if lib_dir not in os.environ.get('LD_LIBRARY_PATH', ''):
            os.environ['LD_LIBRARY_PATH'] = lib_dir + os.pathsep + os.environ.get('LD_LIBRARY_PATH', '')
    except:
        pass

    import torch
    from mmengine.config import Config
    from mmengine.runner import Runner
    print(f"🚀 CUDA Ready: {torch.cuda.is_available()} | Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

## 📂 1. Dataset Preview

Visualize the **satellite imagery**, **instance masks**, and **surface normals**.

In [ ]:
def preview_dataset(data_root, num_samples=3):
    import matplotlib.pyplot as plt
    import numpy as np
    import cv2
    
    data_path = Path(data_root)
    if not data_path.is_absolute():
        data_path = Path(project_root) / data_root
        
    train_file = data_path / 'train.txt'
    if not train_file.exists():
        print(f"❌ Could not find train.txt at {train_file}. Run prepare_omnicity_v2_final.py first!")
        return
        
    with open(train_file, 'r') as f:
        sample_ids = [line.strip() for line in f.readlines()[:num_samples]]
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5 * num_samples))
    for i, sid in enumerate(sample_ids):
        img_path = str(data_path / 'images' / (sid + '.jpg'))
        img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        
        mask_path = str(data_path / 'masks' / (sid + '.png'))
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        mask_vis = cv2.applyColorMap(((mask % 20) * 12).astype(np.uint8), cv2.COLORMAP_JET)
        
        axes[i, 0].imshow(img); axes[i, 0].set_title(sid); axes[i, 0].axis('off')
        axes[i, 1].imshow(mask_vis); axes[i, 1].set_title("Mask"); axes[i, 1].axis('off')
        
        norm_path = data_path / 'normals' / (sid + '.npy')
        if norm_path.exists():
            normals = np.load(str(norm_path))
            axes[i, 2].imshow(((normals + 1) * 127.5).astype(np.uint8))
        axes[i, 2].set_title("Normals"); axes[i, 2].axis('off')
        
    plt.tight_layout(); plt.show()

preview_dataset("data/OmniCity", num_samples=2)

## ⚙️ 2. Training Configuration


In [ ]:
from mmengine.config import Config

MODE = "fine-tune" 
CONFIG_FILE = str(Path(project_root) / "configs/deeproof_finetune_swin_L.py")
WORK_DIR = str(Path(project_root) / "work_dirs/swin_l_omnicity_v2")

cfg = Config.fromfile(CONFIG_FILE)
cfg.work_dir = WORK_DIR
cfg.data_root = str(Path(project_root) / "data/OmniCity/")
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.train_cfg.max_iters = 20000

if MODE == "scratch": cfg.load_from = None
print(f"✅ Configuration Validated. WorkDir: {WORK_DIR}")

## 🚀 3. Start Training


In [ ]:
import torch
from mmengine.runner import Runner

print(f"🚀 Starting Trainer on: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

runner = Runner.from_cfg(cfg)
runner.train()